In [1]:
import sys
import os

sys.path.append('/Users/dan/Code/LLM/llm_analyst')

In [2]:
from llm_analyst.core.config import Config
from llm_analyst.core.research_editor import LLMEditor
from llm_analyst.core.research_analyst import LLMAnalyst
from llm_analyst.core.research_writer import LLMWriter
from llm_analyst.core.research_publisher import LLMPublisher
from llm_analyst.core.prompts import Prompts
from datetime import datetime, timezone
query = "What happened in the latest burning man floods?"
config_params = {}
config_params = {
    "internet_search" :{"default_val":"ddg_search"},
    "llm_provider"    :{"default_val":"openai"},
    "llm_model"       :{"default_val":"gpt-4o-2024-05-13"},
}

#    "llm_model"       :{"default_val":"mixtral-8x7b-32768"},
#    "llm_model"       :{"default_val":"gemma-7b-it"},
#    "llm_model"       :{"default_val":"llama3-8b-8192"},
#    "llm_model"       :{"default_val":"llama3-70b-8192"},
#    "llm_model"       :{"default_val":"gpt-4o-2024-05-13"},


config = Config()
config._set_values_for_config(config_params)


{'internet_search': <function llm_analyst.search_methods.internet_search.ddg_search(query, max_results=5)>,
 'embedding_provider': OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x143f4a9d0>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x143f71910>, model='text-embedding-ada-002', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=set(), disallowed_special='all', chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None),
 'llm_provider': llm_analyst.chat_models.openai.OPENAI_Model,
 'llm_model': 'gpt-4o-

In [3]:
llm_editor = LLMEditor(query, config = config)
detailed_report_state = await llm_editor.create_detailed_report()
print(detailed_report_state.final_report_md)

/Users/dan/miniforge3/envs/gpt-researcher/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


active_research_topic  = What happened in the latest burning man floods?
main_research_topic    = 
Visited URLs length    = 11
Reseach finding length = 4
Report headings length = 0
Markdown report length = 0
agent_type             = 📰 News Agent
agents_role_prompt     = You are a well-informed AI news reporter assistant. Your primary goal is to deliver accurate, detailed, unbiased, and well-structured news reports on recent events, including background information, key developments, and potential implications.

Researching Overview of the Burning Man Floods
Writing Overview of the Burning Man Floods
Researching Impact on Festivalgoers and Festival Operations
Writing Impact on Festivalgoers and Festival Operations
Researching Response and Recovery Efforts
Writing Response and Recovery Efforts
# Catastrophic Floods at Burning Man 2024: A Detailed Report

The Burning Man festival, renowned for its vibrant community and artistic expression, faced unprecedented challenges in 2024 due to sev

In [4]:
await LLMPublisher.init(detailed_report_state).publish_to_pdf_file()

Report written to /Users/dan/llm_analyst_out/bee617f2d6704d39b177464602054eeb.pdf


'/Users/dan/llm_analyst_out/bee617f2d6704d39b177464602054eeb.pdf'

In [ ]:

llm_analyst = LLMAnalyst(query,config = config)
primary_research = choose_agent_dict = await llm_analyst.conduct_research()
print("="*40)
print(primary_research)
subtopics = await llm_analyst.select_subtopics()
print("="*40)
print(subtopics)
report_state = await llm_analyst.write_report()
print("="*40)
print(report_state.report_md)


In [ ]:
from llm_analyst.core.research_writer import LLMWriter

subtopic_reports = []
subtopics_report_body = ""

for subtopic in subtopics:
    print(f"Researching {subtopic}")
    subtopic_assistant = LLMAnalyst(
        active_research_topic = subtopic,
        report_type = "subtopic_report",
        main_research_topic = primary_research.active_research_topic,
        visited_urls = primary_research.visited_urls,
        agents_role_prompt =  primary_research.agents_role_prompt,
        agent_type = primary_research.agent_type
    )
    subtopic_assistant.research_findings = primary_research.research_findings
    subtopic_assistant.report_headings = primary_research.report_headings
     
    subtopic_research = await subtopic_assistant.conduct_research()
    print(f"Writing {subtopic}")
    subtopic_report = await subtopic_assistant.write_report()

    primary_research.research_findings = subtopic_assistant.research_findings
    primary_research.visited_urls.update(subtopic_assistant.visited_urls)
    primary_research.report_headings.append(
        {
            "subtopic task": subtopic_assistant.active_research_topic,
            "headers": LLMWriter.init(subtopic_report).extract_headers()
        }
    )
    subtopic_reports.append({ "topic": subtopic_assistant.active_research_topic, "report": subtopic_report.report_md })
    primary_research.report_md += "\n\n\n" + subtopic_report.report_md

#llm_writer = LLMWriter.init(primary_research)
#introduction = await llm_writer.write_introduction()
    

In [ ]:
introduction = await LLMWriter.init(primary_research).write_introduction()
print(introduction)

In [ ]:
toc = await LLMWriter.init(primary_research).write_table_of_contents()
print(toc)

In [ ]:
references = LLMWriter.init(primary_research).write_references()
print(references)

In [ ]:
import re
prompt_keys = prompts._prompts.keys()
for prompt_key in prompt_keys:
    prompt = prompts.get_prompt(prompt_key)
    variables = re.findall(r'\{(.*?)\}', prompt)
    variables = [f"{variable}={variable}" for variable in list(variables)]
    params = ', '.join(variables)
    fun_call=f"prompts.get_prompt('{prompt_key}',{params})"
    print(fun_call)

    #print("="*20,prompt_key,"="*20)
    #print(f"{prompt_key}({variables})")
    #print(prompt)
    #print("="*80)

In [ ]:
from llm_analyst.core.prompts import Prompts
from datetime import datetime, timezone

context = "This context"
question = "What is the meaning of life?"
total_words = 2000
report_format = "APA"
datetime_now = datetime.now().strftime('%B %d, %Y')

prompts = Prompts()
prompt = prompts.get_prompt("generate_report_prompt",
                            context=context,
                            question=question,
                            total_words=total_words,
                            report_format=report_format,
                            datetime_now=datetime_now)
print(prompt)


In [ ]:
from llm_analyst.core.prompts import Prompts
from datetime import datetime, timezone

context = "This context"
question = "What is the meaning of life?"
total_words = 2000
task = "What is the meaning of life?"
max_iterations = 5
datetime_now = datetime.now().strftime('%B %d, %Y')

prompts = Prompts()
prompt = prompts.get_prompt("generate_resource_report_prompt",
                            context=context,
                            question=question,
                            total_words=total_words)
print(prompt)




In [ ]:

from llm_analyst.core.prompts import Prompts
from datetime import datetime, timezone

prompts = Prompts()
prompt = prompts.get_prompt("auto_agent_instructions")
print(prompt)



In [ ]:
from llm_analyst.core.researcher import LLMAnalyst
query = "What happened in the latest burning man floods?"
llm_analyst = LLMAnalyst(query)
agent, agent_prompt = await llm_analyst.choose_agent()
print(f"Agent={agent}\nAgent Prompt={agent_prompt}")




In [ ]:
import inspect

def my_trace_log(func):
    def wrapper(*args, **kwargs):
        function_name = inspect.currentframe().f_code.co_name
        print(f"TRACE: Entering {func.__name__}")
        result = func(*args, **kwargs)
        print(f"TRACE: Exiting {func.__name__}")
        return result
    return wrapper

@my_trace_log
def hello_world():
    print("Hello World")

# Example usage
hello_world()


In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq
from llm_analyst.core.exceptions import LLMAnalystsException
deterministic_temp=0
role = "You are a well-informed AI news analyst assistant. Your primary goal is to provide comprehensive, accurate, unbiased, and well-structured news reports based on the latest events and developments."
report_prompt = "Information: context data....\n\nUsing the above information, answer the following query or task: 'What happened in the latest burning man floods?' in a detailed report -- The report should focus on the answer to the query, should be well structured, informative, in-depth and comprehensive, with facts and numbers if available and a minimum of 1000 words.\nYou should strive to write the report as long as you can using all relevant and necessary information provided.\nYou must write the report with markdown syntax.\nUse an unbiased and journalistic tone.\nYou MUST determine your own concrete and valid opinion based on the given information. Do NOT deter to general and meaningless conclusions.\nYou MUST write all used source urls at the end of the report as references, and make sure not to add duplicated sources, but only one reference for each.\nEvery url should be hyperlinked: [url website](url)\nAdditionally, you MUST include hyperlinks to the relevant URLs wherever they are referenced in the report : \n\n       e.g.:\n            # Report Header\n\n            This is a sample text. ([url website](url))\n\nYou MUST write the report in APA format.\nCite search results using inline notations.\nOnly cite the most relevant results that answer the query accurately.\nPlace these citations at the end of the sentence or paragraph that reference them.\nPlease do your best, this is very important to my career.\nAssume that the current date is May 21, 2024"
messages=[
        {"role": "system", "content": "role"},
        {"role": "user",   "content": "report_prompt"}]
            
#api_key = os.environ["OPENAI_API_KEY"]
api_key = os.environ["GROQ_API_KEY"]
temperature = 0
#model = "gpt-4o-2024-05-13"
model = "llama3-70b-8192"
max_tokens=4000
llm = ChatGroq(
            model = model,
            temperature = temperature,
            max_tokens = max_tokens,
            api_key = api_key
        )


In [ ]:
config_params = {}
config_params = {
    "internet_search" :{"default_val":"ddg_search"},
    "llm_provider"    :{"default_val":"groq"},
    "llm_model"       :{"default_val":"llama3-8b-8192"},
}

config = Config()
config._set_values_for_config(config_params)
llm_analyst = LLMAnalyst(query,config = config)

In [ ]:
research_state = llm_analyst.get_research_state()
research_state.report_md="Test"
print(research_state)
print(llm_analyst.cfg)

In [ ]:
llm_writer = LLMWriter.init(research_state)
print(llm_writer)

In [12]:
empty_str = ""

if not empty_str:
    print("The string is empty.")
else:
    print("The string is not empty.")


The string is empty.
